In [2]:
!pip install matplotlib-venn

In [3]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [4]:
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 126319 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.4_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.4) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.4) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 12.7 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31629 sha256=cea850ea1e88eaf0cd4bc5ac13cc6dba6d0b53cb247dc86ad11b2d536288f4e0
  Stored in directory: /root/.cache/pip/wheels/32/98/bd/4893d6923dd027f455b250367d402bfd69a6f4416581df46db
Successfully built libarchive


In [5]:
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [6]:
!pip install cartopy
import cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 109.1 MB/s eta 0:00:00


In [9]:
!pip install findspark
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MySparkApp") \
    .getOrCreate()

spark

In [16]:
df = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/Health Care_Patient_survey_source.csv")

In [17]:
#Implement RDD
rdd = df.rdd

In [18]:
#Implement optimizations (CACHE,PARTITIONS) on RDD
rdd.cache()
rdd.repartition(10)

MapPartitionsRDD[19] at coalesce at NativeMethodAccessorImpl.java:0

In [22]:
# Display rdd content in table form
df.show()
df.printSchema()

+-----------+--------------------+--------------------+------+-----+--------+-----------+------------+--------------------+--------------------+--------------------+--------------------------+-----------------------------------+--------------+-----------------------+-----------------+---------------------------+------------------------------------+----------------------------+-------------------------------------+------------------+----------------+--------------------+
|Provider ID|       Hospital Name|             Address|  City|State|ZIP Code|County Name|Phone Number|          Measure ID|            Question|  Answer Description|Patient Survey Star Rating|Patient Survey Star Rating Footnote|Answer Percent|Answer Percent Footnote|Linear Mean Value|Number of Completed Surveys|Number of Completed Surveys Footnote|Survey Response Rate Percent|Survey Response Rate Percent Footnote|Measure Start Date|Measure End Date|            Location|
+-----------+--------------------+----------------

In [30]:
# TASK 7 - List the hospitals which are in same city
from pyspark.sql.functions import collect_set, size

same_city_hospitals = df.groupBy("City") \
    .agg(collect_set("Hospital Name").alias("Hospitals")) \
    .filter(size("Hospitals") > 1).cache()

same_city_hospitals.write.mode("overwrite").partitionBy("City").parquet("same_city_hospitals")

same_city_hospitals.show(truncate=False)


+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|City         |Hospitals                                                                                                                                                                                                                                                                                                |
+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|CONWAY       |[CONWAY REGIONAL MEDICAL CENTER, BAPTIST HE

In [29]:
# TASK 8 - Total Survey Response rate by all the Hospitals.
from pyspark.sql.functions import col, sum as _sum

df_cleaned = df.withColumn(
    "Survey Response Rsate Percent",
    col("Survey Response Rate Percent").cast("double")
)

total_response = df_cleaned.agg(
    _sum("Survey Response Rate Percent").alias("Total Survey Response Rate")
)

total_response.show()

+--------------------------+
|Total Survey Response Rate|
+--------------------------+
|                 1214478.0|
+--------------------------+

